In [93]:
# Daily net generation by balancing authority and energy source. Source: Form EIA-930 Product: Hourly Electric Grid Monitor
# Data link: https://api.eia.gov/v2/electricity/rto/daily-fuel-type-data/data/?frequency=daily&data[0]=value&facets[respondent][]=US48&facets[timezone][]=Eastern&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000

import pandas as pd
import requests # to request/download data via API
from datetime import datetime, timedelta # to break up start & end dates into smaller monthly intervals
from dateutil import rrule # to break up start & end dates into smaller monthly intervals
from dateutil.relativedelta import relativedelta # to break up start & end dates into smaller monthly intervals

# declare variables
start_date = datetime(2019, 11, 1) # start date for downloading data
end_date = datetime(2022, 1, 1) # end date for downloading data
startDates = [] # list to store start dates
endDates = [] # list to store end dates
list_data = [] # list to store data downloaded via API in json format
api_key = 'q6uV6bK4CBECvdpqyeQ6CvldEcQazws5pSJqaw6D'

def API_function(start_date, end_date, list_data):
    # Download raw data using API
    URL = 'https://api.eia.gov/v2/electricity/rto/daily-fuel-type-data/data/?api_key=' + api_key + '&frequency=daily&data[0]=value&facets[respondent][]=US48&facets[timezone][]=Eastern&start=' + start_date.strftime('%Y-%m-%d') + '&end=' + end_date.strftime('%Y-%m-%d') + '&sort[0][column]=period&sort[0][direction]=desc&offset=0&out=json' # convert datetime format (e.g. '2019, 1, 1') to string format (e.g. '2019-01-01') for start and end dates so that these can be understood by this API request
    #URL = 'https://api.eia.gov/series/?series_id=PET.MCRRIP12.M&api_key=q6uV6bK4CBECvdpqyeQ6CvldEcQazws5pSJqaw6D&out=json'
    #URL = 'http://api.eia.gov/category/?api_key=q6uV6bK4CBECvdpqyeQ6CvldEcQazws5pSJqaw6D&category_id=3&out=json'
    #URL = 'https://api.eia.gov/v2/electricity/retail-sales&api_key=q6uV6bK4CBECvdpqyeQ6CvldEcQazws5pSJqaw6D&out=json'
    headers = {'Accept-Encoding': 'identity'}
    r = requests.get(URL, headers=headers)
    json_data = r.json()

    # convert json data format to pandas DataFrame, then extract data as 'temp_list_data' list of dictionaries, then append to 'list_data' list of dictionaries and return it
    df_json = pd.DataFrame(json_data)
    temp_list_data = df_json.at['data', 'response']
    list_data.append(temp_list_data) 
    return list_data

# because each API request is limited to 5000 rows, I need to split the API request into several smaller chunks by creating a code to loop through start & end dates in 1-month increments.
for dt in rrule.rrule(rrule.MONTHLY, dtstart=start_date, until=end_date): # create 'startDates' list and 'endDates' list by looping through start & end dates in monthly increments
    startDates.append(dt) # add start-of-month date to 'startDates' list
    dt = dt + relativedelta(months=+1, days=-1) # calculate end of month date
    endDates.append(dt) # add end-of-month date to 'endDates' list
for start, end in zip(startDates, endDates): # runs 'API_function' function request for each pair of start and end dates stored in 'startDates' and 'endDates' lists and saves the result of each request it to 'list_data' list
    list_data = API_function(start, end, list_data)

# convert data from list of dictionaries format to pandas DataFrame format
col_names = list_data[0][0].keys() # create list of column names as 'col_names' dictionary
col_names = list(col_names) # convert 'col_names' dictionary to list
list_data = [item for sublist in list_data for item in sublist] # flatten 'list_data' list of lists of dictionaries into 'list_date' list of dictionaries in order to be able to parse it in creating a 'df' DataFrame below
df = pd.DataFrame(list_data, columns=col_names) # convert 'list_data' list of dictionaries to 'df' pandas DataFrame

df

,period,respondent,respondent-name,fueltype,type-name,timezone,timezone-description,value,value-units
0,2019-11-30,US48,United States Lower 48,COL,Coal,Eastern,Eastern,1896420,megawatthours
1,2019-11-30,US48,United States Lower 48,SUN,Solar,Eastern,Eastern,94810,megawatthours
2,2019-11-30,US48,United States Lower 48,OTH,Other,Eastern,Eastern,196429,megawatthours
3,2019-11-30,US48,United States Lower 48,NG,Natural gas,Eastern,Eastern,3046173,megawatthours
4,2019-11-30,US48,United States Lower 48,WAT,Hydro,Eastern,Eastern,691028,megawatthours
...,...,...,...,...,...,...,...,...,...
6579,2022-01-01,US48,United States Lower 48,WND,Wind,Eastern,Eastern,1485777,megawatthours
6580,2022-01-01,US48,United States Lower 48,OTH,Other,Eastern,Eastern,184933,megawatthours
6581,2022-01-01,US48,United States Lower 48,SUN,Solar,Eastern,Eastern,195359,megawatthours
6582,2022-01-01,US48,United States Lower 48,WAT,Hydro,Eastern,Eastern,808045,megawatthours


In [57]:
# it's the same code as the code above but for a single API query (i.e. without splitting API into several smaller queries in order to address 5000-row limit)

import pandas as pd
import requests
from datetime import datetime, timedelta # to break up start & end dates into smaller monthly intervals
from dateutil import rrule # to break up start & end dates into smaller monthly intervals
from dateutil.relativedelta import relativedelta # to break up start & end dates into smaller monthly intervals

# https://api.eia.gov/v2/electricity/rto/daily-fuel-type-data/data/?frequency=daily&data[0]=value&facets[respondent][]=US48&facets[timezone][]=Eastern&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000
# Daily net generation by balancing authority and energy source. Source: Form EIA-930 Product: Hourly Electric Grid Monitor

# because each API request is capped at 5000 rows, I need to create a code to loop through start & end dates in 1-month increments & concatenatte the resuld to my DataFrame. See the code in the cell just below this one.
start_date = datetime(2019, 1, 1)
end_date = datetime(2022, 1, 1)

# Download raw data using API
URL = 'https://api.eia.gov/v2/electricity/rto/daily-fuel-type-data/data/?api_key=q6uV6bK4CBECvdpqyeQ6CvldEcQazws5pSJqaw6D&frequency=daily&data[0]=value&facets[respondent][]=US48&facets[timezone][]=Eastern&start=' + start_date.strftime('%Y-%m-%d') + '&end=' + end_date.strftime('%Y-%m-%d') + '&sort[0][column]=period&sort[0][direction]=desc&offset=0&out=json' # convert datetime format (e.g. '2019, 1, 1') to string format (e.g. '2019-01-01') for start and end dates so that these can be understood by this API request
#URL = 'https://api.eia.gov/series/?series_id=PET.MCRRIP12.M&api_key=q6uV6bK4CBECvdpqyeQ6CvldEcQazws5pSJqaw6D&out=json'
#URL = 'http://api.eia.gov/category/?api_key=q6uV6bK4CBECvdpqyeQ6CvldEcQazws5pSJqaw6D&category_id=3&out=json'
#URL = 'https://api.eia.gov/v2/electricity/retail-sales&api_key=q6uV6bK4CBECvdpqyeQ6CvldEcQazws5pSJqaw6D&out=json'
headers = {'Accept-Encoding': 'identity'}
r = requests.get(URL, headers=headers)
#print(r.text[:10000])
# df = pd.DataFrame(r.json())
json_data = r.json()

# convert json data format to pandas DataFrame, then extract data as 'list_data' list of dictionaries
df_json = pd.DataFrame(json_data)
list_data = df_json.at['data', 'response']

# convert data from list of dictionaries format to pandas DataFrame format
col_names = list_data[0].keys() # create list of column names as 'col_names' dictionary
col_names = list(col_names) # convert 'col_names' dictionary to list
df = pd.DataFrame(list_data, columns=col_names) # convert 'list_data' list of dictionaries to 'df' pandas DataFrame
df

,period,respondent,respondent-name,fueltype,type-name,timezone,timezone-description,value,value-units
0,2022-01-01,US48,United States Lower 48,COL,Coal,Eastern,Eastern,1924618,megawatthours
1,2022-01-01,US48,United States Lower 48,WND,Wind,Eastern,Eastern,1485777,megawatthours
2,2022-01-01,US48,United States Lower 48,WAT,Hydro,Eastern,Eastern,808045,megawatthours
3,2022-01-01,US48,United States Lower 48,SUN,Solar,Eastern,Eastern,195359,megawatthours
4,2022-01-01,US48,United States Lower 48,OTH,Other,Eastern,Eastern,184933,megawatthours
...,...,...,...,...,...,...,...,...,...
4995,2020-04-17,US48,United States Lower 48,SUN,Solar,Eastern,Eastern,228341,megawatthours
4996,2020-04-17,US48,United States Lower 48,OTH,Other,Eastern,Eastern,200564,megawatthours
4997,2020-04-17,US48,United States Lower 48,OIL,Petroleum,Eastern,Eastern,9092,megawatthours
4998,2020-04-17,US48,United States Lower 48,NUC,Nuclear,Eastern,Eastern,1991699,megawatthours
